## Lectura del fichero de estado de vacunación publicado por Sanidad en  
* https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm


In [59]:
# importamos pandas, numpy y datetime
# previamente ha sido necesatio instalar el motor odfpy con un pip3 install odfpy
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

Version branch cambios 9 enero

preparamos nombre del fichero csv, dentro del directorio csv. Preparamos dos ficheros:
  * `estado_vacunacion_.csv` , última versión del mismo con el acumulado de todos los informes publicados
  * `estado_vacunacion_yearmonthday.csv`, con la versión del día en cuestión, solo con la información de ese día

In [60]:
path_directorio='csv/'
nombre_root_fichero = 'estado_vacunacion_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=1,day=8)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

Enlace al fichero en red en formato ods

In [61]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

Leemos el fichero

In [62]:
try:
    datos_row = pd.read_excel(enlace)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210114.ods


Le echamos un vistazo para asegurar que ha ido todo bien

In [63]:
datos_row

,Unnamed: 0,Dosis entregadas Pfizer (1),Dosis entregadas Moderna (1),Total Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Fecha de la última vacuna registrada (2)
0,Andalucía,209520,6800,216320,123459,0.570724,2021-01-13
1,Aragón,35415,1200,36615,20854,0.569548,2021-01-13
2,Asturias,35420,1200,36620,29074,0.793938,2021-01-14
3,Baleares,12840,600,13440,7552,0.561905,2021-01-13
4,Canarias,41310,1300,42610,25159,0.590448,2022-01-13
5,Cantabria,17700,500,18200,12779,0.702143,2021-01-14
6,Castilla y Leon,76740,2500,79240,57969,0.731562,2021-01-14
7,Castilla La Mancha,53100,1700,54800,30613,0.558631,2021-01-14
8,Cataluña,180020,5800,185820,105587,0.568222,2021-01-13
9,C. Valenciana,91450,2900,94350,80429,0.852454,2021-01-14


..hacemos una copia para hacerle las transformaciones necesarias:

In [65]:
datos_hoy = datos_row.copy()

Primero renombramos columnas, añadimos luego una columna con la fecha del informe `date_pub`, y finalmente añadimos fuente y origen de datos para asegurar trazabilidad de los datos:

In [66]:
datos_hoy.columns=['ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna','Dosis entregadas','Dosis administradas','% sobre entregadas','Fecha de la ultima vacuna registrada']

In [58]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace


vamos a convertir el formato de `Fecha de la ultima vacuna registrada` para que aparezca como día/mes/año sin añadir ceros de pad..de ahi el signo menos en el caso de dia y mes (%-d/%-m). La excepción se añade para la línea de Totales, que no lleva fecha asociada:

In [37]:
def convierte(x):
    campo =  x['Fecha de la ultima vacuna registrada']
    try:
        fecha = campo.strftime('%-d/%-m/%y')
        return fecha
    except:
        return campo

In [38]:
datos_hoy['Fecha de la ultima vacuna registrada'] = datos_hoy.apply(convierte,axis=1)

..y ordenamos..

In [39]:
datos_hoy=datos_hoy[['date_pub','ccaa','Fecha de la ultima vacuna registrada','Dosis entregadas','Dosis administradas','% sobre entregadas','source_name','source']]

Hemos visto algún informe en el que se añaden espacios al nombre de Comunidad, nos aseguramos aquí de eliminarlos en caso de que vuelvan a aparecer:

In [40]:
datos_hoy['ccaa'] = datos_hoy['ccaa'].apply(lambda x : x.rstrip().lstrip())

In [41]:
datos_hoy['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

In [42]:
datos_hoy

,date_pub,ccaa,Fecha de la ultima vacuna registrada,Dosis entregadas,Dosis administradas,% sobre entregadas,source_name,source
0,13/1/21,Andalucía,12/1/21,209520,109831,0.524203,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,13/1/21,Aragón,12/1/21,35415,17043,0.481237,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,13/1/21,Asturias,13/1/21,35420,25278,0.713665,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,13/1/21,Baleares,12/1/21,12840,4931,0.384034,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,13/1/21,Canarias,13/1/22,41310,22290,0.539579,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,13/1/21,Cantabria,13/1/21,17700,11302,0.638531,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,13/1/21,Castilla y Leon,12/1/21,76740,48243,0.628655,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,13/1/21,Castilla La Mancha,13/1/21,53100,26879,0.506196,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,13/1/21,Cataluña,12/1/21,180020,92133,0.511793,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,13/1/21,C. Valenciana,13/1/21,91450,70870,0.774959,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


guardamos el fichero de hoy en el directorio csv

In [43]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora bajamos el acumulado actual para añadirle los datos de hoy, y volverlo a subir 

In [44]:
# datos_acumulados = pd.read_csv('csv/Libro1.csv',sep=';')

In [45]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [46]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas,Dosis administradas,% sobre entregadas,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,140295,25809,0.183962,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,23715,2004,0.084503,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,23720,9380,0.395447,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,8940,153,0.017114,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,20835,4846,0.232589,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...
115,12/1/21,Navarra,11850,6557,0.553333,NaN,11/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
116,12/1/21,País Vasco,31605,15393,0.487043,NaN,11/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
117,12/1/21,Ceuta,1005,758,0.754229,NaN,11/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
118,12/1/21,Melilla,1005,1005,1.000000,NaN,11/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


concatenamos...

In [47]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [48]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas,Dosis administradas,% sobre entregadas,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,140295,25809,0.183962,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,23715,2004,0.084503,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,23720,9380,0.395447,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,8940,153,0.017114,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,20835,4846,0.232589,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...
135,13/1/21,Navarra,17700,7420,0.419209,NaN,12/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
136,13/1/21,País Vasco,47205,17771,0.376464,NaN,12/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
137,13/1/21,Ceuta,1005,820,0.815920,NaN,12/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
138,13/1/21,Melilla,1005,1005,1.000000,NaN,12/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [49]:
datos_acumulados['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

y volvemos a subirlo..

In [50]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)